In [4]:
import os
import pandas as pd
#!pip install pyarrow
import pyarrow.parquet as pq

In [5]:
os.getcwd()
folder_path = 'taxi'
# Initialize an empty list to store individual DataFrames
dfs = []

# Iterate through each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.parquet'):
        # Construct the full file path
        file_path = os.path.join(folder_path, file_name)
        
        # Read Parquet file into a DataFrame
        df = pq.read_table(file_path).to_pandas()
        
        # Append the DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
trips = pd.concat(dfs, ignore_index=True)

In [6]:
trips.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
Airport_fee                     float64
airport_fee                     float64
dtype: object

In [7]:
trips.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,airport_fee
0,1,2023-06-01 00:08:48,2023-06-01 00:29:41,1.0,3.40,1.0,N,140,238,1,21.9,3.50,0.5,6.70,0.0,1.0,33.60,2.5,0.00,NaN
1,1,2023-06-01 00:15:04,2023-06-01 00:25:18,0.0,3.40,1.0,N,50,151,1,15.6,3.50,0.5,3.00,0.0,1.0,23.60,2.5,0.00,NaN
2,1,2023-06-01 00:48:24,2023-06-01 01:07:07,1.0,10.20,1.0,N,138,97,1,40.8,7.75,0.5,10.00,0.0,1.0,60.05,0.0,1.75,NaN
3,2,2023-06-01 00:54:03,2023-06-01 01:17:29,3.0,9.83,1.0,N,100,244,1,39.4,1.00,0.5,8.88,0.0,1.0,53.28,2.5,0.00,NaN
4,2,2023-06-01 00:18:44,2023-06-01 00:27:18,1.0,1.17,1.0,N,137,234,1,9.3,1.00,0.5,0.72,0.0,1.0,15.02,2.5,0.00,NaN


In [8]:
trips.shape

(38310226, 20)

In [9]:
trips[["tpep_pickup_datetime","tpep_dropoff_datetime"]].max()

tpep_pickup_datetime    2024-01-03 19:42:57
tpep_dropoff_datetime   2024-01-03 20:15:55
dtype: datetime64[ns]

In [10]:
trips[["tpep_pickup_datetime","tpep_dropoff_datetime"]].min()

tpep_pickup_datetime    2001-01-01 00:06:49
tpep_dropoff_datetime   1970-01-20 10:16:32
dtype: datetime64[ns]

In [11]:
trips=  trips[(trips['tpep_dropoff_datetime'].dt.year==2023) & (trips['tpep_pickup_datetime'].dt.year==2023)]

In [15]:
#data cleaning
#1. passanger count
#trips.passenger_count.describe()
trips=trips[(trips['passenger_count']>0)&(trips['passenger_count']<=6)]
#2. trip distance 
trips=trips[(trips['trip_distance']>=0)]
#3. trip fare
#we only focus on RatecodeID=1 which is the Standard Rate, no negotiate prices and other influencer
trips=trips[trips['RatecodeID']==1]
#We only keep orders paid by cash and credit card
trips=trips[(trips['payment_type']==1)|(trips['payment_type']==2)]

In [17]:
trips.groupby('PULocationID').size().sort_values(ascending=False)

PULocationID
237    1686437
161    1631138
236    1487841
162    1261226
138    1234509
        ...   
172          6
176          5
84           5
5            4
27           4
Length: 262, dtype: int64

In [23]:
p_zones = [237, 161, 236, 162,138]
trips=trips[(trips['PULocationID']==237)|(trips['PULocationID']==161)|(trips['PULocationID']==236)|(trips['PULocationID']==162)|(trips['PULocationID']==138)]

In [20]:
start_date_2023 = pd.to_datetime('2023-01-01 00:00')
end_date_2023 = pd.to_datetime('2024-01-01 00:00')
timeline_2023 = pd.date_range(start=start_date_2023, end=end_date_2023, freq='30min')


In [ ]:
result = pd.DataFrame(columns=['tpep_pickup_datetime', 'order_cnt', 'PULocationID'])
for idx, time in enumerate(timeline_2023[:-1]):
    next_time = timeline_2023[idx + 1]
    orders_in_interval = trips[(trips['tpep_pickup_datetime'] >= time) & (df['tpep_pickup_datetime'] < next_time)]
    order_counts = orders_in_interval.groupby('PULocationID').size().reset_index(name='order_cnt')
    order_counts['tpep_pickup_datetime'] = time
    result = pd.concat([result, order_counts], ignore_index=True)
    if(idx%10==0):
        print(f"{idx}done")

/var/folders/0x/k389_vt16p1gjts_tn1qjff40000gn/T/ipykernel_1202/1482100882.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  orders_in_interval = trips[(trips['tpep_pickup_datetime'] >= time) & (df['tpep_pickup_datetime'] < next_time)]


0done


/var/folders/0x/k389_vt16p1gjts_tn1qjff40000gn/T/ipykernel_1202/1482100882.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  orders_in_interval = trips[(trips['tpep_pickup_datetime'] >= time) & (df['tpep_pickup_datetime'] < next_time)]
/var/folders/0x/k389_vt16p1gjts_tn1qjff40000gn/T/ipykernel_1202/1482100882.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  orders_in_interval = trips[(trips['tpep_pickup_datetime'] >= time) & (df['tpep_pickup_datetime'] < next_time)]
/var/folders/0x/k389_vt16p1gjts_tn1qjff40000gn/T/ipykernel_1202/1482100882.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  orders_in_interval = trips[(trips['tpep_pickup_datetime'] >= time) & (df['tpep_pickup_datetime'] < next_time)]
/var/folders/0x/k389_vt16p1gjts_tn1qjff40000gn/T/ipykernel_1202/1482100882.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  orders_in_interval = trips[(tri

10done


/var/folders/0x/k389_vt16p1gjts_tn1qjff40000gn/T/ipykernel_1202/1482100882.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  orders_in_interval = trips[(trips['tpep_pickup_datetime'] >= time) & (df['tpep_pickup_datetime'] < next_time)]
/var/folders/0x/k389_vt16p1gjts_tn1qjff40000gn/T/ipykernel_1202/1482100882.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  orders_in_interval = trips[(trips['tpep_pickup_datetime'] >= time) & (df['tpep_pickup_datetime'] < next_time)]
/var/folders/0x/k389_vt16p1gjts_tn1qjff40000gn/T/ipykernel_1202/1482100882.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  orders_in_interval = trips[(trips['tpep_pickup_datetime'] >= time) & (df['tpep_pickup_datetime'] < next_time)]
/var/folders/0x/k389_vt16p1gjts_tn1qjff40000gn/T/ipykernel_1202/1482100882.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  orders_in_interval = trips[(tri

...

/var/folders/0x/k389_vt16p1gjts_tn1qjff40000gn/T/ipykernel_1202/1482100882.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  orders_in_interval = trips[(trips['tpep_pickup_datetime'] >= time) & (df['tpep_pickup_datetime'] < next_time)]
/var/folders/0x/k389_vt16p1gjts_tn1qjff40000gn/T/ipykernel_1202/1482100882.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  orders_in_interval = trips[(trips['tpep_pickup_datetime'] >= time) & (df['tpep_pickup_datetime'] < next_time)]
/var/folders/0x/k389_vt16p1gjts_tn1qjff40000gn/T/ipykernel_1202/1482100882.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  orders_in_interval = trips[(trips['tpep_pickup_datetime'] >= time) & (df['tpep_pickup_datetime'] < next_time)]
/var/folders/0x/k389_vt16p1gjts_tn1qjff40000gn/T/ipykernel_1202/1482100882.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  orders_in_interval = trips[(tri

1260done


/var/folders/0x/k389_vt16p1gjts_tn1qjff40000gn/T/ipykernel_1202/1482100882.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  orders_in_interval = trips[(trips['tpep_pickup_datetime'] >= time) & (df['tpep_pickup_datetime'] < next_time)]
/var/folders/0x/k389_vt16p1gjts_tn1qjff40000gn/T/ipykernel_1202/1482100882.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  orders_in_interval = trips[(trips['tpep_pickup_datetime'] >= time) & (df['tpep_pickup_datetime'] < next_time)]
/var/folders/0x/k389_vt16p1gjts_tn1qjff40000gn/T/ipykernel_1202/1482100882.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  orders_in_interval = trips[(trips['tpep_pickup_datetime'] >= time) & (df['tpep_pickup_datetime'] < next_time)]
/var/folders/0x/k389_vt16p1gjts_tn1qjff40000gn/T/ipykernel_1202/1482100882.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  orders_in_interval = trips[(tri

1270done


/var/folders/0x/k389_vt16p1gjts_tn1qjff40000gn/T/ipykernel_1202/1482100882.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  orders_in_interval = trips[(trips['tpep_pickup_datetime'] >= time) & (df['tpep_pickup_datetime'] < next_time)]
